In [169]:
import cv2
import numpy as np
import dlib
from prediction import *

In [277]:
path = './eyebrow/4.png'

face_origin = cv2.imread(path)
face_copy = face_origin.copy()
face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)

detector = dlib.get_frontal_face_detector()
dlib_rects = detector(face_rgb, 1)

# img_how_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)

predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

img = []

for dlib_rect in dlib_rects:
    points = predictor(face_rgb, dlib_rect)
    list_points = list(map(lambda p: [p.x, p.y], points.parts()))
    img.append(list_points)
    img = np.array(img[0])
    
xs = img[22:27,0]
ys = img[22:27,1]

x = min(xs)
y = min(ys)
w = max(xs) - min(xs)
h = max(ys) - min(ys)

crop_img = face_copy[y:y+h, x:x+w]
bw = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
invert_img = cv2.bitwise_not(bw)
binary_image = cv2.adaptiveThreshold(invert_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                    cv2.THRESH_BINARY, 49, 2)

In [278]:
cv2.imshow('threshold', binary_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [279]:
DEFAULT = 64
if binary_image.shape[0] > binary_image.shape[1]:
        DEFAULT = binary_image.shape[0]
else:
    DEFAULT = binary_image.shape[1]

BG_COLOR = (0, 0, 0)

height, width = binary_image.shape

w_x = int((DEFAULT - (width)) / 2)
h_y = int((DEFAULT - (height)) / 2)

img_pad = cv2.copyMakeBorder(binary_image, h_y, h_y, w_x, w_x, cv2.BORDER_CONSTANT, value=[0, 0, 0])

w_x2 = DEFAULT - img_pad.shape[1]
h_y2 = DEFAULT - img_pad.shape[0]

img_pad = cv2.copyMakeBorder(img_pad, h_y2, 0, w_x2, 0, cv2.BORDER_CONSTANT, value=[0, 0, 0])
img_pad = cv2.resize(img_pad, (64, 64))
cv2.imshow("padding", img_pad)
cv2.waitKey(0)
cv2.destroyAllWindows()